In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# from langchain_community.vectorstores import Qdrant
# from langchain_openai.embeddings import OpenAIEmbeddings
# from langchain_text_splitters import TokenTextSplitter

In [ ]:
import os
import glob
import re
import json
from langchain.schema import Document

In [8]:
# Specify the directory and file pattern
directory = "data/codigo_penal/text/"
file_pattern = "*.txt"
metadata_file = "metadata.json" 

with open(os.path.join(directory, metadata_file), "r") as f:
    metadata = json.load(f)

# Find all .txt files in the directory
file_list = sorted(glob.glob(f"{directory}/{file_pattern}"), key=lambda x: int(re.search(r"\d+", x).group()))

docs = []
# Iterate over each file
for file_path in file_list:
    # Read the file content
    with open(file_path, "r") as file:
        content = file.read()

    # Replace 3 or more consecutive newlines with 2 newlines
    docs.append(Document(page_content=content, metadata={...}))

In [ ]:
import glob
import re
import json
from langchain.schema import Document

# Specify the directory and file pattern
directory = "data/codigo_penal/text/"
file_pattern = "*.txt"
metadata_file = "metadata.json"  # Path to the metadata.json file

# Load the metadata from the JSON file
with open(metadata_file, "r") as f:
    metadata = json.load(f)

# Find all .txt files in the directory
file_list = sorted(glob.glob(f"{directory}/{file_pattern}"), key=lambda x: int(re.search(r"\d+", x).group()))

docs = []
# Iterate over each file
for file_path in file_list:
    # Extract the filename from the file_path
    filename = file_path.split('/')[-1]

    # Read the file content
    with open(file_path, "r") as file:
        content = file.read()

    # Replace 3 or more consecutive newlines with 2 newlines
    content = re.sub(r'\n{3,}', '\n\n', content)

    # Retrieve the corresponding metadata using the filename
    file_metadata = {"source_url": metadata.get(filename)}

    # Create a Document object with the content and metadata
    docs.append(Document(page_content=content, metadata=file_metadata))


In [9]:
from langchain_text_splitters import CharacterTextSplitter, TokenTextSplitter
import tiktoken

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4-turbo-preview").encode(
        text,
    )
    return len(tokens)

text_splitter = CharacterTextSplitter(
    separator="\n\n\n",
    chunk_size=100,
    chunk_overlap=0,
    keep_separator=False,
    length_function=tiktoken_len,
    is_separator_regex=False,
)

# text_splitter = TokenTextSplitter(
#     chunk_size=512,
#     chunk_overlap=32,
# )
splitted_docs = text_splitter.split_documents(documents=docs)

Created a chunk of size 720, which is longer than the specified 100
Created a chunk of size 154, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 198, which is longer than the specified 100
Created a chunk of size 108, which is longer than the specified 100
Created a chunk of size 106, which is longer than the specified 100
Created a chunk of size 225, which is longer than the specified 100
Created a chunk of size 308, which is longer than the specified 100
Created a chunk of size 1189, which is longer than the specified 100
Created a chunk of size 265, which is longer than the specified 100
Created a chunk of size 231, which is longer than the specified 100
Created a chunk of size 164, which is longer than the specified 100
Created a chunk of size 132, which is longer than the specified 100
Created a chunk of size 344, which is longer than the specified 100
Created a chunk of size 191, which is longer th

In [5]:
# from IPython.display import display, Markdown
# for idx, each in enumerate(splitted_docs, start=1):
#     display(Markdown(f"#### Chunk {idx}"))
#     display(Markdown(each.page_content))
#     if idx == 3:
#         break

In [10]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [11]:
import os
qdrant = Qdrant.from_documents(
    documents=splitted_docs,
    embedding=embeddings,
    url="https://40b3acb8-a18f-47c2-b60d-13fab390785d.us-east4-0.gcp.cloud.qdrant.io:6333",
    api_key=os.getenv("QDRANT__API_KEY"),
    prefer_grpc=True,
    collection_name="my_documents",
    force_recreate=False,
)

In [11]:
query = "ANTIJURIDICIDAD"
found_docs = qdrant.similarity_search(query, k=5)

In [12]:
from IPython.display import display, Markdown
for idx, each in enumerate(found_docs, start=1):
    display(Markdown(f"### Index ---- {idx}\n\n"))
    display(Markdown(each.page_content))

### Index ---- 1



ARTÍCULO 11. ANTIJURIDICIDAD. Para que una conducta típica sea punible se requiere que lesione o ponga efectivamente en peligro, sin justa causa, el bien jurídicamente tutelado por la ley penal.

### Index ---- 2



ARTÍCULO 6o. LEGALIDAD. Nadie podrá ser juzgado sino conforme a las leyes preexistentes al acto que se le imputa, ante el juez o tribunal competente y con la observancia de la plenitud de las formas propias de cada juicio. La preexistencia de la norma también se aplica para el reenvío en materia de tipos penales en blanco. 
La ley permisiva o favorable, aun cuando sea posterior se aplicará, sin excepción, de preferencia a la restrictiva o desfavorable. Ello también rige para los condenados. 
La analogía sólo se aplicará en materias permisivas.

### Index ---- 3



ARTÍCULO 228. IMPUTACIONES DE LITIGANTES. <Artículo CONDICIONALMENTE exequible> Las injurias expresadas por los litigantes, apoderados o defensores en los escritos, discursos o informes producidos ante los tribunales y no dados por sus autores a la publicidad, quedarán sujetas únicamente a las correcciones y acciones disciplinarias correspondientes. 
Jurisprudencia Vigencia
Corte Constitucional- Artículo declarado EXEQUIBLE, por los cargos examinados, por la Corte Constitucional mediante Sentencia C-442-11 de 25 de mayo 2011, Magistrado Ponente Dr. Humberto Antonio Sierra Porto.- Artículo declarado EXEQUIBLE por la Corte Constitucional mediante Sentencia C-392-02 de 22 de mayo de 2002, Magistrado Ponente Dr. Alvaro Tafur Galvis; 'bajo el entendido que las injurias eximidas de sanción penal son solamente aquellas que guardan relación de causalidad con el objeto del proceso'. 
TÍTULO VI. 
DELITOS CONTRA LA FAMILIA 
CAPÍTULO I. 
DE LA VIOLENCIA INTRAFAMILIAR

### Index ---- 4



ARTÍCULO 224. EXIMENTE DE RESPONSABILIDAD. No será responsable de las conductas descritas en los artículos anteriores quien probare la veracidad de las imputaciones. 
Sin embargo, en ningún caso se admitirá prueba: 
1. <Numeral INEXEQUIBLE>
Jurisprudencia Vigencia
Corte Constitucional- Numeral declarado INEXEQUIBLE por la Corte Constitucional mediante Sentencia C-417-09 de 26 de junio de 2009, Magistrado Ponente Dr. Juan Carlos Henao Pérez. Destaca el Editor:'Esta decisión empero, no debe entenderse como base para restar de majestad a la justicia, para reducir la fuerza imperativa derivada de la cosa juzgada y, como se ha dicho, para minar la seguridad y confianza en el derecho creado por los jueces al impartir justicia. Todos estos valores siguen siendo esenciales en el Estado constitucional del Derecho y una sociedad democrática y respetuosa de los derechos y libertades, requiere de la existencia y garantía de tales instituciones. Tampoco se trata de sentar una doctrina que avala una suerte de “dictadura de los periodistas”, privilegiados sin límites de expresarse e informar sin responsabilidad ninguna y sin soporte razonable en la realidad, pues un tal ejercicio de esas libertades preferentes, sería contrario a la lógica y armonía que requiere el sistema de derechos y en general, el orden constitucional. El fundamento único y exclusivo de esta decisión se encuentra sólo en el hecho de que la medida allí contenida ha resultado ser innecesaria y desproporcionada respecto de la libertad de expresión del art. 20 de la Constitución.' 
Legislación Anterior
Texto original de la Ley 599 de 2000:1. Sobre la imputación de cualquier conducta punible que hubiere sido objeto de sentencia absolutoria, preclusión de la investigación o cesación de procedimiento o sus equivalentes, excepto si se tratare de prescripción de la acción, y  
2. <Numeral CONDICIONALMENTE exequible> Sobre la imputación de conductas que se refieran a la vida sexual, conyugal, marital o de familia, o al sujeto pasivo de un delito contra la libertad y la formación sexuales. 
Jurisprudencia Vigencia
Corte Constitucional- Numeral 2 declarado CONDICIONALMENTE EXEQUIBLE por la Corte Constitucional mediante Sentencia C-222-22 de 23 de junio de 2022, Magistrada Ponente Dra. Cristina Pardo Schlesinger. 'en el entendido de que la excepción de veracidad podrá ser aplicable como excluyente de responsabilidad cuando las imputaciones de conductas que se refieran a la vida sexual, conyugal, marital o de familia, o al sujeto pasivo de delitos contra la libertad, integridad y formación sexuales, tengan interés público y cuenten con el consentimiento de la víctima, por los cargos analizados en esta sentencia.'. 
Jurisprudencia Vigencia
Corte Constitucional- Artículo declarado EXEQUIBLE, por los cargos examinados, por la Corte Constitucional mediante Sentencia C-442-11 de 25 de mayo 2011, Magistrado Ponente Dr. Humberto Antonio Sierra Porto.- La Corte Constitucional se declaró INHIBIDA de fallar sobre este artículo (parcial) por ineptitud de la demanda, mediante Sentencia C-401-07 de 23 de mayo de 2007, Magistrado Ponente Dr. Manuel José Cepeda Espinosa.

### Index ---- 5



ARTÍCULO 227. INJURIAS O CALUMNIAS RECIPROCAS. Si las imputaciones o agravios a que se refieren los artículos 220, 221 y 226 fueren recíprocas, se podrán declarar exentos de responsabilidad a los injuriantes o calumniantes o a cualquiera de ellos. 
Jurisprudencia Vigencia
Corte Constitucional- Artículo declarado EXEQUIBLE, por los cargos examinados, por la Corte Constitucional mediante Sentencia C-442-11 de 25 de mayo 2011, Magistrado Ponente Dr. Humberto Antonio Sierra Porto.

In [13]:
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [14]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")
llm = ChatOpenAI(model="gpt-4-turbo-preview")
document_chain = create_stuff_documents_chain(llm, prompt)

In [15]:
from langchain.chains import create_retrieval_chain
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

retriever = qdrant.as_retriever()
retrieval_chain = {"input": RunnablePassthrough()} | create_retrieval_chain(retriever, document_chain) | itemgetter("answer")

In [90]:
Markdown((llm | StrOutputParser()).invoke("Que es la territorialidad?"))

La territorialidad es un concepto amplio que se aplica en varias disciplinas, incluyendo la psicología, la sociología, la antropología, la geografía, y el derecho, entre otras. Aunque tiene distintas connotaciones en cada campo, en general, la territorialidad se refiere a la manera en que los individuos y los grupos establecen control, influencia o dominio sobre un espacio o territorio específico. Este concepto implica no solo la ocupación física de un espacio, sino también la delimitación de ese espacio a través de límites simbólicos o legales, la defensa del mismo contra intrusos, y la personalización o modificación del espacio para reflejar la identidad o las necesidades de quien lo controla.

En términos biológicos, muchos animales exhiben comportamiento territorial, marcando y defendiendo su territorio contra otros individuos de su especie como una manera de asegurar recursos como alimento y lugares de anidación, así como para establecer dominio social o reproductivo.

En el ámbito humano, la territorialidad puede manifestarse de muchas maneras, desde la propiedad personal de bienes (como una casa o un coche) hasta la ocupación y control de grandes extensiones de tierra por parte de estados o naciones. También se observa en contextos más pequeños, como la disposición de los espacios en oficinas o la selección de asientos en espacios públicos, reflejando deseos de privacidad, control y comunicación social.

En el derecho, la territorialidad es un principio fundamental que se refiere al alcance de las leyes y la jurisdicción de un estado dentro de sus fronteras geográficas. Este principio implica que cada estado tiene soberanía para legislar y aplicar sus leyes dentro de su territorio.

La territorialidad, por tanto, es un concepto multifacético que juega un papel crucial en la organización de las sociedades humanas y de muchas otras especies, influenciando las interacciones sociales, económicas, políticas y culturales.

In [16]:
Markdown(retrieval_chain.invoke("Que es la territorialidad?"))

La territorialidad, según el contexto proporcionado, se refiere al principio según el cual la ley penal colombiana se aplica a toda persona que infrinja la ley dentro del territorio nacional. Esto implica que cualquier delito cometido dentro de las fronteras de Colombia está sujeto a la jurisdicción y a las leyes penales colombianas, a excepción de las excepciones que puedan estar consagradas en el derecho internacional. Este principio se extiende a delitos cometidos a bordo de naves o aeronaves del Estado colombiano o explotadas por este, que se encuentren fuera del territorio nacional, siempre que no haya excepciones establecidas por tratados o convenios internacionales ratificados por Colombia.